In [ ]:
import matplotlib.pyplot as pp
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tp

from functools import partial

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
pp.style.use('ggplot')

# A/B testing

In [ ]:
def unnormalized_log_probability(success_num_a, success_num_b,
                                 total_num_a, total_num_b,
                                 rate_a, rate_b):
    rv_rate_a = tp.distributions.Uniform()
    rv_rate_b = tp.distributions.Uniform()
    rv_success_a = tp.distributions.Bernoulli(probs=rate_a)
    rv_success_b = tp.distributions.Bernoulli(probs=rate_b)
    return (
        rv_rate_a.log_prob(rate_a) +
        rv_rate_b.log_prob(rate_b) +
        tf.to_float(success_num_a) * rv_success_a.log_prob(1) +
        tf.to_float(success_num_b) * rv_success_b.log_prob(1) +
        tf.to_float(total_num_a - success_num_a) * rv_success_a.log_prob(0) +
        tf.to_float(total_num_b - success_num_b) * rv_success_b.log_prob(0)
    )

rate_a = 0.04
rate_b = 0.05
total_num = 100000
success_num_a = tf.reduce_sum(tp.distributions.Bernoulli(probs=rate_a).sample(sample_shape=total_num))
success_num_b = tf.reduce_sum(tp.distributions.Bernoulli(probs=rate_b).sample(sample_shape=total_num))

state = [
    tf.to_float(success_num_a / total_num),
    tf.to_float(success_num_b / total_num),
]

bijector = [
    tp.bijectors.Sigmoid(),
    tp.bijectors.Sigmoid(),
]

with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    step_size = tf.get_variable(
        initializer=tf.constant(0.5, dtype=tf.float32),
        name='step_size',
        trainable=False,
        use_resource=True,
    )

kernel = tp.mcmc.TransformedTransitionKernel(
    inner_kernel=tp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=partial(unnormalized_log_probability,
                                   success_num_a, success_num_b,
                                   total_num, total_num),
        num_leapfrog_steps=3,
        step_size=step_size,
        step_size_update_fn=tp.mcmc.make_simple_step_size_update_policy(),
        state_gradients_are_stopped=True,
    ),
    bijector=bijector,
)

[posterior_rate_a, posterior_rate_b], kernel = tp.mcmc.sample_chain(
    num_results=100000,
    num_burnin_steps=10000,
    current_state=state,
    kernel=kernel,
)

In [ ]:
session = tf.Session()

session.run([
    tf.global_variables_initializer(),
    tf.local_variables_initializer(),
])

[
    posterior_rate_a_,
    posterior_rate_b_,
    kernel_,
] = session.run([
    posterior_rate_a,
    posterior_rate_b,
    kernel,
])

print('Acceptance rate: {}'.format(kernel_.inner_results.is_accepted.mean()))

In [ ]:
pp.figure(figsize=(12, 6))
pp.axvline(x=rate_b - rate_a, color='black', linestyle='--', lw=1)
pp.hist(posterior_rate_b_ - posterior_rate_a_, bins=50, density=True);

# Challenger shuttle

In [ ]:
tf.reset_default_graph()

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/master/Chapter2_MorePyMC/data/challenger_data.csv')
data = data.rename(columns={'Damage Incident': 'Incident'})[['Temperature', 'Incident']]
data = data[data['Incident'].isin(['0', '1'])].astype('float32')

In [ ]:
def logistic(x, a, b):
    return 1.0 / (1.0 + tf.exp(a * x + b))

def unnormalized_log_probability(data, a, b):
    temperature = tf.convert_to_tensor(data['Temperature'])
    incident = tf.convert_to_tensor(data['Incident'])
    prior_a = tp.distributions.Normal(loc=0.0, scale=1000.0)
    prior_b = tp.distributions.Normal(loc=0.0, scale=1000.0)
    probability = logistic(temperature, a, b)
    failure = tp.distributions.Bernoulli(probs=probability)
    return (
        prior_a.log_prob(a) +
        prior_b.log_prob(b) +
        tf.reduce_sum(failure.log_prob(incident))
    )

state = [
    0.0,
    0.0,
]

bijector = [
    tp.bijectors.Identity(),
    tp.bijectors.Identity(),
]

with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    step_size = tf.get_variable(
        initializer=tf.constant(0.5, dtype=tf.float32),
        name='step_size',
        trainable=False,
        use_resource=True,
    )

kernel = tp.mcmc.TransformedTransitionKernel(
    inner_kernel=tp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=partial(unnormalized_log_probability, data),
        num_leapfrog_steps=2,
        step_size=step_size,
        step_size_update_fn=tp.mcmc.make_simple_step_size_update_policy(),
        state_gradients_are_stopped=True,
    ),
    bijector=bijector,
)

[posterior_a, posterior_b], kernel = tp.mcmc.sample_chain(
    num_results=100000,
    num_burnin_steps=90000,
    current_state=state,
    kernel=kernel,
)

In [ ]:
session = tf.Session()

session.run([
    tf.global_variables_initializer(),
    tf.local_variables_initializer(),
])

[
    posterior_a_,
    posterior_b_,
    kernel_,
] = session.run([
    posterior_a,
    posterior_b,
    kernel,
])

print('Acceptance rate: {}'.format(kernel_.inner_results.is_accepted.mean()))

In [ ]:
pp.figure(figsize=(12, 6))
pp.subplot(211)
pp.hist(posterior_a_, bins=35, density=True)
pp.subplot(212)
pp.hist(posterior_b_, bins=35, density=True);

In [ ]:
probability = logistic(31, posterior_a_, posterior_b_)
probability_ = session.run(probability)

In [ ]:
pp.figure(figsize=(12, 6))
pp.hist(probability_, bins=100);